### Testing notebook

In [1]:
import requests
import urllib3
import os
import pandas as pd
import math
import pprint
import json
import time
import sys
from multiprocessing import Pool
from PIL import Image

In [45]:
def bing_search(query, count=8, offset=0):
    url = 'https://api.cognitive.microsoft.com/bing/v5.0/images/search'
    # Query string parameters.
    payload = {
        'q': query,
        'count': count,
        'offset': offset,
        'modulesRequested': 'all'
    }
    # Custom headers.
    headers = {
        'Content-Type': 'multipart/form-data',
        'Ocp-Apim-Subscription-Key': 'cd2f8800f47840fda58b777410b41a7e'
    }
    try:
        response = requests.post(url, params=payload, headers=headers)
    except:
        print "damn!"
    return response.json()

In [46]:
def dataFrame(result):
    link_list = [item['contentUrl'] for item in result]
    df = pd.DataFrame(link_list, columns = ["contentUrl"])
    df["name"] = [item['name'] for item in result]
    df["encodingFormat"] = [item['encodingFormat'] for item in result]
    df["insights"] = [item['imageInsightsToken'] for item in result]
    df["imageId"] = [item['imageId'] for item in result]
    return df

In [47]:
def bing_search_adv(imageId, insightsToken):
    url = 'https://api.cognitive.microsoft.com/bing/v5.0/images/search'
    # Query string parameters.
    payload = {
        'id': imageId,
        'insightsToken': insightsToken,
        'modulesRequested': 'all'
    }
    headers = {
        'Content-Type': 'multipart/form-data',
        'Ocp-Apim-Subscription-Key': 'cd2f8800f47840fda58b777410b41a7e'
    }
    # Make GET requests.
    response = requests.post(url, params=payload, headers=headers)
    # Get JSON response.
    return response.json()

#### test: 

In [48]:
query = 'herpes zoster'
result = bing_search(query, count=1)
dFrame = dataFrame(result.get('value', {}))

In [49]:
print result.keys()

[u'instrumentation', u'_type', u'displayRecipeSourcesBadges', u'value', u'webSearchUrl', u'nextOffsetAddCount', u'displayShoppingSourcesBadges', u'totalEstimatedMatches']


In [50]:
result

{u'_type': u'Images',
 u'displayRecipeSourcesBadges': True,
 u'displayShoppingSourcesBadges': False,
 u'instrumentation': {u'pageLoadPingUrl': u'https://www.bingapis.com/api/ping/pageload?IG=96E591D049FE4FDC848ABA1C892E8123&CID=10BCA1F07A09631903F6AADC7BC962AB&Type=Event.CPT&DATA=0'},
 u'nextOffsetAddCount': 0,
 u'totalEstimatedMatches': 100,
 u'value': [{u'accentColor': u'386593',
   u'contentSize': u'622120 B',
   u'contentUrl': u'http://www.bing.com/cr?IG=96E591D049FE4FDC848ABA1C892E8123&CID=10BCA1F07A09631903F6AADC7BC962AB&rd=1&h=ECXxjAJyX_tFTBfKruyN4j-HDgF4g4sKDoNirdB4cLw&v=1&r=http%3a%2f%2fwww.regionalderm.com%2fRegional_Derm%2fRD_Large%2fHerpes_zoster_necrotic9.jpg&p=DevEx,5009.1',
   u'datePublished': u'2017-10-14T00:04:00',
   u'encodingFormat': u'jpeg',
   u'height': 1169,
   u'hostPageDisplayUrl': u'regionalderm.com/Regional_Derm/files/herpes_zoster.html',
   u'hostPageUrl': u'http://www.bing.com/cr?IG=96E591D049FE4FDC848ABA1C892E8123&CID=10BCA1F07A09631903F6AADC7BC962AB&rd=

In [51]:
dFrame

,contentUrl,name,encodingFormat,insights,imageId
0,http://www.bing.com/cr?IG=96E591D049FE4FDC848A...,HERPES ZOSTER,jpeg,ccid_/nqhiF2K*mid_60FA6D412C9C35AA050C150A2609...,60FA6D412C9C35AA050C150A26093E71D6EFCC4E


In [52]:
# for i in range(dFrame.shape[0]):
# print "imageId: " + dFrame.iloc[i, 4]
# print "insights: " + dFrame.iloc[i, 3] + "\n"
i = 0
result_i = bing_search_adv(imageId = dFrame.iloc[i, 4], insightsToken = dFrame.iloc[i, 3])

for key in result_i.keys():
    print "key: " + key
    print result_i[key]
    print "\n"
    if key == 'visuallySimilarImages':
        for similarImg in result_i[key]:
            print similarImg
            print "\n"

key: instrumentation
{u'pageLoadPingUrl': u'https://www.bingapis.com/api/ping/pageload?IG=B3235687C28D43B087283CBFE938CACB&CID=2E5EEF4C86346BDE06AAE46087F46A01&Type=Event.CPT&DATA=0'}


key: _type
ImageInsights


key: categoryClassification
{u'categories': [{u'name': u'food'}, {u'name': u'zit'}, {u'name': u'skin lesions'}, {u'name': u'skin conditions'}, {u'name': u'treatment'}, {u'name': u'meat'}, {u'name': u'skin cancer'}, {u'name': u'skin tags'}, {u'name': u'acne treatment'}, {u'name': u'skin rash'}]}


key: visuallySimilarImages
[{u'contentUrl': u'http://www.bing.com/cr?IG=B3235687C28D43B087283CBFE938CACB&CID=2E5EEF4C86346BDE06AAE46087F46A01&rd=1&h=DnF_NcAmg89ayo00UBaEcm1BGSjlgXpoVH1u1oTUAl0&v=1&r=http%3a%2f%2fthumbs.imagekind.com%2f4602027_650%2fMixed-media--by-rafi-talby_art.jpg%3fv%3d1492732183&p=DevEx,5022.1', u'accentColor': u'9B3042', u'height': 488, u'hostPageDisplayUrl': u'www.imagekind.com/art/stunning/mixed-media/artwork-on...', u'name': u'Stunning "Mixed Media" Artwork Fo